In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "C:/Users/Jan/Dropbox/_Programmieren/Udemy ML Kurs/13 - Case Study/3 - Case Study Income/1 - Lösung/adult.xlsx"
df = pd.read_excel(path)

In [3]:
print(df.columns)

Index(['age', 'workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'gender', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


In [4]:
print(df.dtypes)

age                int64
workclass         object
education         object
marital-status    object
occupation        object
relationship      object
race              object
gender            object
hours-per-week     int64
native-country    object
income            object
dtype: object


In [5]:
print(df.shape, df.size)

(48842, 11) 537262


In [6]:
df.head()

,age,workclass,education,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,<=50K
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,>50K
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States,>50K
4,18,?,Some-college,Never-married,?,Own-child,White,Female,30,United-States,<=50K


In [7]:
for col in df.columns:
    print(col, " Number of categories: ", len(df[col].value_counts()))

age  Number of categories:  74
workclass  Number of categories:  9
education  Number of categories:  16
marital-status  Number of categories:  7
occupation  Number of categories:  15
relationship  Number of categories:  6
race  Number of categories:  5
gender  Number of categories:  2
hours-per-week  Number of categories:  96
native-country  Number of categories:  42
income  Number of categories:  2


In [8]:
data = df.as_matrix()
x = data[:,:-1] # Gehe bis zur letzten Spalte (die aber nicht einschließlich)
y = data[:,-1] # Nimm nur die letzte Spalte

print(x.shape, y.shape)

# <=50K: Klasse 0
# >50K: Klasse 1
y = np.array([0 if val == "<=50K" else 1 for val in y], dtype=np.int8)

print(y)
x[:,[0,9]] = x[:,[9,0]]
print(x[0])

(48842, 10) (48842,)
[0 0 1 ... 0 0 1]
['United-States' 'Private' '11th' 'Never-married' 'Machine-op-inspct'
 'Own-child' 'Black' 'Male' 40 25]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [23]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [16]:
# Race: White, Black, Asian
#         0       1     2
#     [1,0,0]  [0,1,0]  [0,0,1]
# Kateogrie White = Kategorie 0 => [1, 0, 0]

In [17]:
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(x[:,:-2])

x_cat = enc.transform(x[:,:-2]).toarray()
x_cat = np.append(x_cat, x[:,-2:], axis=1)
print(x_cat.shape)
print(x_cat[0])

(48842, 104)
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 40 25]


In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_cat, y, test_size=0.3)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [24]:
parameters = {"n_estimators": [50, 100, 200], "max_depth": [None, 30, 50, 100]}
clf = RandomForestClassifier(n_jobs=-1)
clf = GridSearchCV(clf, parameters, cv=3)
clf.fit(x_train, y_train)

print("Best parameters set found on developement set:")
print(clf.best_params_, "\n")

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std*2, params))

Best parameters set found on developement set:
{'max_depth': 30, 'n_estimators': 50} 

0.821 (+/-0.002) for {'max_depth': None, 'n_estimators': 50}
0.819 (+/-0.006) for {'max_depth': None, 'n_estimators': 100}
0.821 (+/-0.005) for {'max_depth': None, 'n_estimators': 200}
0.828 (+/-0.002) for {'max_depth': 30, 'n_estimators': 50}
0.828 (+/-0.002) for {'max_depth': 30, 'n_estimators': 100}
0.828 (+/-0.002) for {'max_depth': 30, 'n_estimators': 200}
0.820 (+/-0.002) for {'max_depth': 50, 'n_estimators': 50}
0.820 (+/-0.004) for {'max_depth': 50, 'n_estimators': 100}
0.820 (+/-0.003) for {'max_depth': 50, 'n_estimators': 200}
0.819 (+/-0.002) for {'max_depth': 100, 'n_estimators': 50}
0.819 (+/-0.004) for {'max_depth': 100, 'n_estimators': 100}
0.820 (+/-0.003) for {'max_depth': 100, 'n_estimators': 200}


In [26]:
parameters = {"kernel": ["sigmoid", "poly", "rbf"]}
clf = SVC()
clf = GridSearchCV(clf, parameters, cv=3)
clf.fit(x_train, y_train)

print("Best parameters set found on developement set:")
print(clf.best_params_, "\n")

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std*2, params))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarni

Best parameters set found on developement set:
{'kernel': 'rbf'} 

0.816 (+/-0.000) for {'kernel': 'sigmoid'}
0.826 (+/-0.006) for {'kernel': 'poly'}
0.834 (+/-0.008) for {'kernel': 'rbf'}


In [29]:
from keras.layers import *
from keras.models import *
from keras.optimizers import *

In [38]:
# Klassifikation:
# >= 2 Klassen: OneHotEncoding für die y-Werte, am Ende des NN Softmax, loss categorical_crossentropy
# = 2 Klassen: y-Werte haben den binären Wert der Klasse (0, 1), am Ende des NN Sigmoid ~(0, 1), loss binary_crossentropy

input_shape = x_train.shape[1] # 104
output_shape = 1

model = Sequential()
model.add(Dense(128, input_dim=input_shape)) # Input layer
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(output_shape))
model.add(Activation("sigmoid")) # Output layer, sigmoid (0, 1) => Wahrscheinlichkeit dafür, dass der Datenpunkt zur Klasse 1 gehört

optimizer = SGD(lr=0.001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10, validation_data=[x_test, y_test])

Train on 34189 samples, validate on 14653 samples
Epoch 1/10
34189/34189 [==============================] - 2s 50us/step - loss: 0.5549 - acc: 0.7446 - val_loss: 0.5057 - val_acc: 0.7641
Epoch 2/10
34189/34189 [==============================] - 1s 38us/step - loss: 0.4758 - acc: 0.7725 - val_loss: 0.4540 - val_acc: 0.7850
Epoch 3/10
34189/34189 [==============================] - 1s 37us/step - loss: 0.4328 - acc: 0.8020 - val_loss: 0.4215 - val_acc: 0.8099
Epoch 4/10
34189/34189 [==============================] - 1s 37us/step - loss: 0.4049 - acc: 0.8179 - val_loss: 0.4005 - val_acc: 0.8213
Epoch 5/10
34189/34189 [==============================] - 1s 37us/step - loss: 0.3867 - acc: 0.8251 - val_loss: 0.3873 - val_acc: 0.8243
Epoch 6/10
34189/34189 [==============================] - 1s 37us/step - loss: 0.3748 - acc: 0.8282 - val_loss: 0.3789 - val_acc: 0.8262
Epoch 7/10
34189/34189 [==============================] - 1s 39us/step - loss: 0.3670 - acc: 0.8302 - val_loss: 0.3735 - val_acc